# **2D MT inversion**
**Elaborado por:** \
Ivan Camilo Caballero Jaimes 2210623\
Johan Enrique Páez Romero 2210631\
Semillero de Investigación en Geofísica Aplicada y Computacional - **SIGAC**

## Librerías

In [2]:
import matplotlib.pyplot as plt
import scipy.sparse as sp
import numpy as np
import unittest
from scipy.constants import mu_0
from discretize.tests import check_derivative
import discretize
import matplotlib.patheffects as pe
from SimPEG.electromagnetics import natural_source as nsem
from SimPEG.electromagnetics.static import utils as sutils
from SimPEG import (
    maps, utils, optimization, objective_function, inversion, inverse_problem, directives,
    data_misfit, regularization, data
)
from discretize import TensorMesh
## ERROR-----> 
from pymatsolver import Solver
# Antes -> "import Pardiso"
# Corregir -> "import Solver"

from scipy.spatial import cKDTree
from scipy.stats import norm
# from dask.distributed import Client, LocalCluster
import dill
from geoana.em.fdem import skin_depth
import discretize.utils as dis_utils
import warnings
warnings.filterwarnings("ignore")

## Parámetros de Inversión y Errores

In [ ]:
input_data = dill.load(open("./synthetic_2d.pik", "rb"))
nskip_freq = 7
nskip_rx = 3
# a decent setup
# nskip_freq = 7
# nskip_rx = 3
relative_error =  0.05
floor_error = 2
alpha_s = 1.#1e-5
alpha_y = 1.#0.2
alpha_z = 1.
rho_0 = 100.
maxIter = 30
use_irls = True
p_s=0.
p_y=2.
p_z=2.

rx_locs = input_data['rx_locs'][::nskip_rx,:]
frequencies = input_data['frequencies'][::nskip_freq]
app_rho_te = input_data['app_rho_te'][::nskip_freq,::nskip_rx]
app_rho_tm = input_data['app_rho_tm'][::nskip_freq,::nskip_rx]
phase_te = input_data['phase_te'][::nskip_freq,::nskip_rx]
phase_tm = input_data['phase_tm'][::nskip_freq,::nskip_rx]


app_rho_err_tm = abs(app_rho_tm) * relative_error
app_rho_err_te = abs (app_rho_te) * relative_error
phase_err_tm = np.ones_like(phase_tm) * floor_error
phase_err_te = np.ones_like(phase_te) * floor_error


## Crear el Modelo Mesh

In [3]:
def generate_2d_mesh_for_mt( # Parámetros
    rx_locs, #Ubicación de los receptores
    frequencies, #Frecuencias de la señal
    sigma_background,  # Conductividad de fondo
    z_factor_max=5, # Los demas son factores y límites para configuar las dimensiones de las celdas del mesh
    z_factor_min=5,
    pfz_down = 1.2,
    pfz_up = 1.5,
    npadz_up = 5,
    x_factor_max = 2,
    spacing_factor = 4,
    pfx = 1.5,
    n_max=1000
):
    # Cálculo de la dimensión en profundidad -------
    f_min = frequencies.min() # Frecuencia mínima
    f_max = frequencies.max() # Frecuencia máxima
    dz_min = np.round(skin_depth(f_max, sigma_background) / z_factor_max) # Tamaño mínimo de celda en profundidad, basado en la profunidad de penetración de la señal (skin depth)
    lz = skin_depth(sigma_background, f_min) * z_factor_max # Longitud en la dirección z

    # Setting the domain length in z-direction
    for nz_down in range(n_max): # Ajusta "hz_down" hasta que la suma de las celdas en "hz_down" supera "lz"
        hz_down = dz_min*pfz_down**np.arange(nz_down)[::-1]
        if hz_down.sum()>lz:
            break
    hz_up = [(dz_min, npadz_up, pfz_up)]
    hz_up = dis_utils.unpack_widths(hz_up)
    hz = np.r_[hz_down, hz_up]   

    # Cálculo de la dimensión lateral -----
    d_station = np.diff(rx_locs[:,0]).min() # Distancia mínima entre estaciones receptoras
    dx_min = np.round(d_station/spacing_factor) # Tamaño mínimo de celda en la dirección lateral
    lx = rx_locs[:,0].max() - rx_locs[:,0].min() # Longitud en la dirección x
    ncx = int(lx / dx_min)
    lx_pad = skin_depth(sigma_background, f_min) * x_factor_max # Longitud de padding en la dirección x
    for npadx in range(n_max): # Ajusta "hx_pad" hasta que la suma de las celdas en "hz_pad" supera "lx_pad"
        hx_pad = dis_utils.unpack_widths([(dx_min, npadx, -pfx)])
        if hx_pad.sum()>lx_pad:
            break
    hx = [(dx_min, npadx, -pfx), (dx_min, ncx), (dx_min, npadx, pfx)]  
    
    mesh = discretize.TensorMesh([hx, hz])
    mesh.origin = np.r_[-mesh.h[0][:npadx].sum()+rx_locs[:,0].min(), -hz_down.sum()]   
    print (mesh)
    return mesh

In [ ]:
mesh = generate_2d_mesh_for_mt(
    rx_locs, 
    frequencies, 
    1e-2)
ind_active = mesh.cell_centers[:,1]<0.

print ("Rx")
print (rx_locs)
print ("Freqs")
print (frequencies)

In [ ]:
ax = mesh.plot_grid()
ax.set_xlim(-50000, 370000)
ax.set_ylim(-200000, 5000)
ax.plot(rx_locs[:, 0], rx_locs[:, 1], "rv")

## Crear Fuentes y Receptores

In [5]:
# simulation class for TM mode
rx_list_tm = [
    nsem.receivers.PointNaturalSource(
        rx_locs, orientation="xy", component="apparent_resistivity"
    ),
    nsem.receivers.PointNaturalSource(
        rx_locs, orientation="xy", component="phase"
    ),
]
src_list_tm = [nsem.sources.Planewave(rx_list_tm, frequency=f) for f in frequencies]
survey_tm = nsem.Survey(src_list_tm)

act_map = maps.InjectActiveCells(mesh, ind_active, np.log(1e-8))
exp_map = maps.ExpMap(mesh=mesh)
sigma_map = exp_map * act_map

sim_tm= nsem.simulation.Simulation2DElectricField(
    mesh,
    survey=survey_tm,
    sigmaMap=sigma_map,
    solver=Solver,
)

# simulation class for TE mode
rx_list_te = [
    nsem.receivers.PointNaturalSource(
        rx_locs, orientation="yx", component="apparent_resistivity"
    ),
    nsem.receivers.PointNaturalSource(
        rx_locs, orientation="yx", component="phase"
    ),
]
src_list_te = [nsem.sources.Planewave(rx_list_te, frequency=f) for f in frequencies]
survey_te = nsem.Survey(src_list_te)

sim_te = nsem.simulation.Simulation2DMagneticField(
    mesh,
    survey=survey_te,
    sigmaMap=sigma_map,
    solver=Solver,
)

In [6]:
dobs_te = np.hstack((app_rho_te, phase_te)).flatten()
dobs_tm = np.hstack((app_rho_tm, phase_tm)).flatten()

std_te = np.hstack((app_rho_err_te, phase_err_te)).flatten()
std_tm = np.hstack((app_rho_err_tm, phase_err_tm)).flatten()

In [7]:
print(len(rx_locs)*2*len(frequencies))
print(len(dobs_te))
print(len(phase_te))


54
54
3


In [11]:
len(phase_te)
app_rho_te

array([[ 89.44342851,  72.82257135,  44.98446484,  14.20548847,
          4.98805002,   7.92523267,  37.14096652,  71.90550965,
         77.79436939],
       [ 96.0897691 , 111.8366866 , 107.91650797, 101.04109534,
         69.04644337,  72.64518741, 114.76137581, 104.65139439,
         96.97962583],
       [106.22813118, 100.13668108,  97.18973656,  99.10094543,
         99.71488127,  88.07614064, 101.17019197, 112.56427609,
         99.18672871]])

In [31]:
m0 = np.ones(ind_active.sum()) * np.log(1./rho_0)

## Inversion

In [32]:
te_data_object = data.Data(survey_te, dobs=dobs_te, standard_deviation=std_te) 
tm_data_object = data.Data(survey_tm, dobs=dobs_tm, standard_deviation=std_tm) 
dmis_te = data_misfit.L2DataMisfit(data=te_data_object, simulation=sim_te)
dmis_tm = data_misfit.L2DataMisfit(data=tm_data_object, simulation=sim_tm)
dmis = dmis_te + dmis_tm

# Define the regularization (model objective function)
reg = regularization.Sparse(
    mesh,
    indActive=ind_active,
    # ERROR -> 
    reference_model=m0,
    # Antes -> mref=m0
    # Corregir -> reference_model=m0
    alpha_s=alpha_s,
    alpha_x=alpha_y,
    alpha_y=alpha_z,
    mapping=maps.IdentityMap(nP=int(ind_active.sum()))
)

# Define how the optimization problem is solved. Here we will use an
# Inexact Gauss Newton approach.
opt = optimization.InexactGaussNewton(maxIter=maxIter, maxIterCG=30)

# Here we define the inverse problem that is to be solved
inv_prob = inverse_problem.BaseInvProblem(dmis, reg, opt)

#######################################################################
# Define MT Inversion Directives
# ------------------------------
#
# Here we define any directives that are carried out during the inversion. This
# includes the cooling schedule for the trade-off parameter (beta), stopping
# criteria for the inversion and saving inversion results at each iteration.
#

# Defining a starting value for the trade-off parameter (beta) between the data
# misfit and the regularization.
starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=1)

# Set the rate of reduction in trade-off parameter (beta) each time the
# the inverse problem is solved. And set the number of Gauss-Newton iterations
# for each trade-off paramter value.
beta_schedule = directives.BetaSchedule(coolingFactor=2, coolingRate=1)

save_dictionary = directives.SaveOutputDictEveryIteration()
save_dictionary.outDict = {}

# Setting a stopping criteria for the inversion.
target_misfit = directives.TargetMisfit(chifact=1)

if use_irls:
    # ERROR --->
    reg.norms = np.r_[p_s, p_y, p_z]
    # Antes -> np.c_
    # Corregir -> np.r_

    # Reach target misfit for L2 solution, then use IRLS until model stops changing.
    IRLS = directives.Update_IRLS(max_irls_iterations=40, minGNiter=1, f_min_change=1e-5)

    # The directives are defined as a list.
    directives_list = [
        IRLS,
        starting_beta,
        save_dictionary,
    ]
else:
    directives_list = [
        starting_beta,
        beta_schedule,
        save_dictionary,
        target_misfit,
    ]

    
#####################################################################
# Running the MT Inversion
# ------------------------
#
# To define the inversion object, we need to define the inversion problem and
# the set of directives. We can then run the inversion.
#

# Here we combine the inverse problem and the set of directives
mt_inversion = inversion.BaseInversion(inv_prob, directiveList=directives_list)

# Run inversion
recovered_conductivity_model = mt_inversion.run(m0)


                        SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
                        ***Done using same Solver, and solver_opts as the Simulation2DMagneticField problem***
                        
model has any nan: 0
============================ Inexact Gauss Newton ============================
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
-----------------------------------------------------------------------------
x0 has any nan: 0
   0  4.14e-06  1.35e+05  0.00e+00  1.35e+05    2.89e+04      0              
   1  2.07e-06  7.38e+03  1.16e+09  9.78e+03    2.09e+03      0              
   2  1.04e-06  5.98e+02  1.41e+09  2.06e+03    1.58e+02      0   Skip BFGS  
   3  5.18e-07  2.65e+02  1.08e+09  8.22e+02    1.15e+02      0              
   4  2.59e-07  1.84e+02  6.10e+08  3.42e+02    7.17e+01      0              
   5  1.30e-07  1.03e+02  7.16e+08  1.95e+02    6.80e+01      0              
   6  6.48e-08  9.87e+01  6

## Visualización

In [23]:
output_dict = save_dictionary.outDict
target_misfit = (dobs_te.size+dobs_tm.size) / 2.
iterations = list(output_dict.keys())
n_iteration = len(iterations)
phi_ds = np.zeros(n_iteration)
phi_ms = np.zeros(n_iteration)
betas = np.zeros(n_iteration)
for ii, iteration in enumerate(iterations):
    phi_ds[ii] = output_dict[iteration]['phi_d']
    phi_ms[ii] = output_dict[iteration]['phi_m']
    betas[ii] = output_dict[iteration]['beta']

In [24]:
import matplotlib
matplotlib.rcParams['font.size'] = 14
def tikhonov_curve(iteration, scale='log'):
    fig, ax = plt.subplots(1,1, figsize=(5,5))
    ax.plot(phi_ms, phi_ds)
    ax.plot(phi_ms[iteration-1], phi_ds[iteration-1], 'ro')
    ax.set_xlabel("$\phi_m$")
    ax.set_ylabel("$\phi_d$")
    if scale == 'log':
        ax.set_xscale('log')
        ax.set_yscale('log')
    xlim = ax.get_xlim()
    ax.plot(xlim, np.ones(2) * target_misfit, '--')
    ax.set_title("Iteration={:d}, Beta = {:.1e}".format(iteration, betas[iteration-1]))
    ax.set_xlim(xlim)
    plt.show()

In [25]:
from ipywidgets import interact, widgets
Q_iter = interact(
    tikhonov_curve, 
    iteration=widgets.IntSlider(min=1, max=int(n_iteration), value=n_iteration),
    scale=widgets.RadioButtons(options=['linear', 'log'])
)

interactive(children=(IntSlider(value=30, description='iteration', max=30, min=1), RadioButtons(description='s…

In [26]:
from matplotlib.colors import LogNorm
mesh_true = input_data['mesh']
sigma_true = input_data['sigma']
iteration = Q_iter.widget.kwargs['iteration']
m = output_dict[iteration]['m']

In [27]:
def foo_model(iteration):
    fig, axs = plt.subplots(2,1, figsize=(10, 6))
    ax, ax1 = axs
    m = output_dict[iteration]['m']
    sigma = np.ones(mesh.nC) * 1e-8
    sigma[ind_active] = np.exp(m)
    sigma_min = 1e-3
    sigma_max = 10
    grid= False
    out = mesh.plot_image(
        sigma, grid=grid, ax=ax, pcolor_opts={'norm':LogNorm(vmin=sigma_min, vmax=sigma_max), 'cmap':'turbo'},
        range_x=(-60000, 60000), range_y=(-30000, 0)
    )
    cb = plt.colorbar(out[0], fraction=0.01, ax=ax)
    cb.set_label("Conductivity (S/m)")
    ax.set_aspect(1)
    ax.set_xlabel("Easting (m)")
    ax.set_ylabel("Elevation (m)")
    ax.plot(rx_locs[:,0], rx_locs[:,1], 'ro')
    out = mesh_true.plot_image(
        sigma_true, grid=grid, ax=ax1, pcolor_opts={'norm':LogNorm(vmin=sigma_min, vmax=sigma_max), 'cmap':'turbo'},
        range_x=(-60000, 60000), range_y=(-30000, 0)
    )
    cb = plt.colorbar(out[0], fraction=0.01, ax=ax1)
    cb.set_label("Conductivity (S/m)")
    ax1.set_aspect(1)
    ax1.set_xlabel("Easting (m)")
    ax1.set_ylabel("Elevation (m)")

In [28]:
Q_misfit = interact(
    foo_model, 
    iteration=widgets.IntSlider(min=1, max=n_iteration, value=iteration),
)    

interactive(children=(IntSlider(value=30, description='iteration', max=30, min=1), Output()), _dom_classes=('w…

In [21]:
n_freq = len(frequencies)
n_rx = rx_locs.shape[0]

In [22]:
PRED_te = pred_te.reshape((n_freq, 2, n_rx))
PRED_tm = pred_tm.reshape((n_freq, 2, n_rx))

rho_app_te_pred = PRED_te[:,0,:]
rho_app_tm_pred = PRED_tm[:,0,:]

phase_te_pred = PRED_te[:,1,:]
phase_tm_pred = PRED_tm[:,1,:]

DOBS_te = dobs_te.reshape((n_freq, 2, n_rx))
DOBS_tm = dobs_tm.reshape((n_freq, 2, n_rx))

rho_app_te_dobs = DOBS_te[:,0,:]
rho_app_tm_dobs = DOBS_tm[:,0,:]

phase_te_dobs = DOBS_te[:,1,:]
phase_tm_dobs = DOBS_tm[:,1,:]

In [23]:
def foo_pred(irx):
    import matplotlib
    matplotlib.rcParams['font.size'] = 10

    fig, axs = plt.subplots(4,1, figsize=(8, 10))
    ax1, ax2, ax3, ax4 = axs
    ax1.loglog(1./frequencies, rho_app_te_pred[:, irx], color='C0')
    ax1.loglog(1./frequencies, rho_app_te_dobs[:, irx], 'x', color='C0')
    ax2.loglog(1./frequencies, rho_app_tm_pred[:, irx], color='C1')
    ax2.loglog(1./frequencies, rho_app_tm_dobs[:, irx], 'x', color='C1')
    for ax in axs[:2]:
        ax.set_ylim(1, 1000)
        ax.set_ylabel("App. Res. (Ohm-m)")
        ax.grid(which='both', alpha=0.3)
    ax3.semilogx(1./frequencies, phase_te_pred[:, irx], color='C0')
    ax3.semilogx(1./frequencies, phase_te_dobs[:, irx], 'x', color='C0')
    ax4.semilogx(1./frequencies, phase_tm_pred[:, irx]+180, color='C1')
    ax4.semilogx(1./frequencies, phase_tm_dobs[:, irx]+180, 'x', color='C1')
    for ax in axs[2:]:
        ax.set_ylim(0, 90)
        ax.set_ylabel("Phase (degree)")
        ax.grid(which='both', alpha=0.3)
    ax4.set_xlabel("Period (s)")

In [24]:
Q_misfit = interact(
    foo_pred, 
    irx=widgets.IntSlider(min=0, max=int(n_rx)-1, value=0),
)

interactive(children=(IntSlider(value=0, description='irx', max=8), Output()), _dom_classes=('widget-interact'…